In [ ]:
import pickle
import os
import sys
working_dir = os.getcwd()
os.chdir("../../..")
sys.path.append(os.getcwd())
from Internal_Representation.state import State
from Solver.action_tracker import ActionTracker
from runner import Runner
os.chdir(working_dir)

# Operational Tools

In [ ]:
# Load file

In [ ]:
# Load pickled Actions_taken

In [ ]:
# Load pickled final state

# Running operations on collected Data

In [ ]:
# Load files